# COPT and LOLP Calculator

## import and settting printer

In [ ]:
import itertools
import math

import numpy as np
import pandas as pd

from tqdm.auto import tqdm

In [ ]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [ ]:
# NOTE:
#   UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
#   COMMENT IF USING MANUAL INPUT
# csv input path
df = pd.read_csv('../data/COPT Case.csv')

# extract csv
capacities = df['Capacity (MW)'].tolist()
outage_rates = df['FOR (Outage)'].tolist()
status = [True] * len(df.index)

In [ ]:
# # NOTE:
# #   UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
# #   COMMENT IF USING CSV INPUT
# # max capacity
# capacities = [
#     80,
#     70,
#     40,
#     50,
#     40,
#     50
# ]

# # forced outage rate
# outage_rates = [
#     0.1, 
#     0.2,
#     0.1,
#     0.2,
#     0.1,
#     0.1
# ]


# # status of generator,
# #     True: exist
# #     False: not exist
# status = [
#     True,
#     False,
#     True,
#     True,
#     True,
#     True,
# ]

In [ ]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,50,0.100,True
2,50,0.100,True
3,50,0.100,True
4,30,0.100,True
5,10,0.070,True
6,20,0.070,True
7,10,0.100,True
8,5,0.100,True
9,10,0.070,True
10,10,0.070,True


In [ ]:
# peak load or load at time considering
# used to calculate LOLP
demands = [590, 470, 585, 650, 660]

## Parameters

In [ ]:
# NOTE:
#   There is two batch, bathc in evaluating of permutation and batch in making COPT.

BATCH_PRODUCT = 100  # batch size for each making table before combine duplicate
MIN_COVERAGE = 1  # early stopper
BATCH_GENERATOR = 10

# TODO:
#   1. Implement resample, useful for big COPT table, triggered only if max table length achieved

## COPT table

In [ ]:
generator_list = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]

last_generator = 0
n_generator = len(capacities)

tables = []
while last_generator < n_generator:
    try:
        ls = generator_list[last_generator:last_generator+BATCH_GENERATOR]
    except:
        ls = generator_list[last_generator:]
    last_generator += BATCH_GENERATOR

    capacities_fix = [[cap, 0] for cap, _ in ls]
    probabilities_fix = [[1-out, out] for _, out in ls]

    capacities_combination_generator = itertools.product(*capacities_fix)
    probabilities_combinations_generator = itertools.product(*probabilities_fix)

    table = []
    with tqdm(total=MIN_COVERAGE) as pbar:
        sum_probability = 0
        while sum_probability <= MIN_COVERAGE:
            table += [[sum(i), np.prod(j)] for i, j in itertools.islice(zip(capacities_combination_generator,
                                                                            probabilities_combinations_generator),
                                                                        BATCH_PRODUCT)]

            # change table to array
            table = np.array(table)

            # sort table
            table = table[(-table[:, 0]).argsort(),:]

            # combine duplicate
            table = [[k,sum([x[1] for x in list(g)])] 
                    for k,g in itertools.groupby(table, lambda x:x[0])]

            # progress bar
            sum_probability = np.sum(table, axis=0)[1]
            pbar.update(n=sum_probability - pbar.n)

    # add all units off
    if table[-1][0] != 0:
        table = np.vstack([table, [0, 1 - sum_probability]])
    else:
        table = np.array(table)

    # add cumulative probability
    table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
    table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

    # save table to tables
    tables.append((table)) 

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

In [ ]:
# TODO: combine tables
table = tables[0].copy()  # copy to avoid modifying data
for table_ in tables[1:]:
    # faster than flatten + transpose
    table = np.hstack(((table[:, 0, None] + table_[:, 0]).reshape(-1,1),
                       (table[:, 1, None] * table_[:, 1]).reshape(-1,1)))

    # sort table
    table = table[(-table[:, 0]).argsort(),:]

    # combine duplicate
    table = np.array([[k,sum([x[1] for x in list(g)])] 
                    for k,g in itertools.groupby(table, lambda x:x[0])])

    # add cumulative probability
    table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
    table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

In [ ]:
columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1)).to_csv('../results/merged.csv')

## LOLP

In [ ]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [ ]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
lolp

[0.00019334443587659762,
 1.4799527806098756e-07,
 0.0001449366283941024,
 0.0032895996560149167,
 0.0050267079116176495]

In [ ]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.0087
